# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [20]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings
from IPython.display import clear_output

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
import dask.dataframe as dd

In [4]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## TaggedDocument For Doc2Vec

In [6]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Load Data

#### Path

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
    outcome_predata = '/Volumes/disk1/pre_data_for_comments/'
    outcome_tagged_data = '/Volumes/disk1/pre_data_for_comments2/'
    outcome_vectorized_data = '/Volumes/disk1/pre_data_for_comments3/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'
    outcome_predata = 'd:/pre_data_for_comments/'
    outcome_tagged_data = 'd:/pre_data_for_comments2/'
    outcome_vectorized_data = 'd:/pre_data_for_comments3/'

#### News 

In [8]:
os.listdir(news_senti_outcome)

['naver_news_sentiment_analysis.csv', 'daum_news_sentiment_analysis.csv']

In [9]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
print (reNaverData.shape)
reNaverData.head()

(15015, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1465,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","['외무성', '핵전쟁', '대변인']","{'미국', '조선반도', '핵전쟁', '북한', '중앙', '도화선', '고위',...",86.0,22.0,negative,Naver
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2062,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"['예산', '예결위', 'soc']","{'호남', '예산안', '의원', '정부안', '증액', '지역구', '국민의당'}",46.0,62.0,positive,Naver
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,536,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"['최경환', '구속영장', '국가정보원']","{'구속영장 청구', '조사', '의원', '정기국회', '국정원장', '검찰', ...",77.0,31.0,negative,Naver
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,133,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"['이슈 · 최재형 감사원장 내정', '감사원장', '최재형', '감사원']","{'공직 사회', '법관', '생활', '감사원장', '지명', '후보자'}",39.0,69.0,positive,Naver
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,932,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"['김정은', 'b-1b', '한반도']","{'공장', '양강도', '사무차장', '삼지연', '시찰', '훈련', '접경',...",72.0,36.0,negative,Naver


In [10]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]
print (reDaumData.shape)
reDaumData.head()

(9369, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'건강', '부작용', '공장', '남성들', '지역', '세보 효과', '연기'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'건강', '커피', '자세', '치아', '철분', '디스크', '점심 식사',...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'앙심', '남편', '성관계', '주변', '진술', '지인', '범행', '아...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","['신영자', '갑질', '롯데백화점']","{'네이처리퍼블릭', '롯데', '징역', '유통업체', '신영자 이사장', '검찰...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'폭행', '혐의', '과자', '범행', '쪽지', '남자친구', '경찰', '...",102.0,6.0,negative,daum


### 뉴스별 댓글에 따른 분석 진행
* 데스크탑에서 첫번째 파일로 진행을 하니 jupyter notebook이 죽어버림

#### Load Model

In [11]:
taggerName_ct = 'ct'
print ('{}'.format(taggerName_ct))
model1_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
name1_ct = '-'.join(re.split('[\(\),\/]',str(model1_ct)))+taggerName_ct
print ('Load model')
model2_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
name2_ct = '-'.join(re.split('[\(\),\/]',str(model2_ct)))+taggerName_ct
print ('Load model')
model3_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')
name3_ct = '-'.join(re.split('[\(\),\/]',str(model3_ct)))+taggerName_ct
print ('Load model')

ct
Load model
Load model
Load model


In [12]:
taggerName_mecab = 'mecab'
print ('{}'.format(taggerName_mecab))
model1_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-mecab.model')
name1_mecab = '-'.join(re.split('[\(\),\/]',str(model1_mecab)))+taggerName_mecab
print ('Load model')
model2_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-mecab.model')
name2_mecab = '-'.join(re.split('[\(\),\/]',str(model2_mecab)))+taggerName_mecab
print ('Load model')
model3_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-mecab.model')
name3_mecab = '-'.join(re.split('[\(\),\/]',str(model3_mecab)))+taggerName_mecab
print ('Load model')

mecab
Load model
Load model
Load model


#### Load Classifier

In [13]:
classifier_by_model1_ct = glob(classifierPath+'*'+name1_ct)
load_Classifier_by_model1_ct = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model1_ct))

classifier_by_model2_ct = glob(classifierPath+'*'+name2_ct)
load_Classifier_by_model2_ct = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model2_ct))

classifier_by_model3_ct = glob(classifierPath+'*'+name3_ct)
load_Classifier_by_model3_ct = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model3_ct))

classifier_by_model1_mecab = glob(classifierPath+'*'+name1_mecab)
load_Classifier_by_model1_mecab = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model1_mecab))

classifier_by_model2_mecab = glob(classifierPath+'*'+name2_mecab)
load_Classifier_by_model2_mecab = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model2_mecab))

classifier_by_model3_mecab = glob(classifierPath+'*'+name3_mecab)
load_Classifier_by_model3_mecab = dict(map(lambda x:bm.LoadClassifier(x), classifier_by_model3_mecab))

In [14]:
def PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token, model, classifierDict, outcomeFileName):
    import pandas as pd
    from glob import glob
    import Basic_Module as bm
    import os
    inferVector_by_model = bm.Get_Infer_Vector(token, model)
    predict_Outcome = dict(map(lambda x: bm.PredictSentiment(inferVector_by_model, x, classifierDict[x]), classifierDict))
    predict_Outcome = pd.DataFrame.from_dict(predict_Outcome)
    predict_Outcome = extCommentsDf.merge(predict_Outcome,
                                   left_index = True, right_index = True)
    predict_Outcome.to_csv(outcomeFileName, index=None, encoding='utf-8')
    

In [22]:
warnings.filterwarnings('ignore')
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')
for idx in reNaverData.index:
    clear_output()
    data = reNaverData.loc[idx]
    newsId = data.name
    print (newsId)
    siteInfo = data.site
    if os.path.isdir(os.path.join(outcomeNaverCommentsPath, newsId)):
        outcomePath = os.path.join(outcomeNaverCommentsPath, newsId)
    else:
        os.mkdir(os.path.join(outcomeNaverCommentsPath, newsId))
        outcomePath = os.path.join(outcomeNaverCommentsPath, newsId)
    commentsFile = naverCommentsPath + newsId + '.csv'
    commentsDf = pd.read_csv(commentsFile, encoding='utf-8', header=0)
    commentsDf.dropna(inplace = True)
    extCommentsDf = commentsDf.loc[:,['category', 'comments', 'date', 'rank', 'site', '공감', '비공감']]
    # twitter
    token_ct = commentsDf.apply(lambda x: bm.TokenizeAndTag(ct, x, stopwords, TaggedDocument), axis = 1)
    outcomeFileName1_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name1_ct))
    if not os.path.isfile(outcomeFileName1_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model1_ct,
                                     load_Classifier_by_model1_ct,
                                     outcomeFileName1_ct)
    outcomeFileName2_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name2_ct))
    if not os.path.isfile(outcomeFileName2_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model2_ct,
                                     load_Classifier_by_model2_ct,
                                     outcomeFileName2_ct)
    outcomeFileName3_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name3_ct))
    if not os.path.isfile(outcomeFileName3_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model3_ct,
                                     load_Classifier_by_model3_ct,
                                     outcomeFileName3_ct)
    del token_ct
    # mecab
    token_mecab = commentsDf.apply(lambda x: bm.TokenizeAndTag(mecab, x, stopwords, TaggedDocument), axis = 1)
    outcomeFileName1_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name1_mecab))
    if not os.path.isfile(outcomeFileName1_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model1_mecab,
                                     load_Classifier_by_model1_mecab,
                                     outcomeFileName1_mecab)
    outcomeFileName2_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name2_mecab))
    if not os.path.isfile(outcomeFileName2_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model2_mecab,
                                     load_Classifier_by_model2_mecab,
                                     outcomeFileName2_mecab)
    outcomeFileName3_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name3_mecab))
    if not os.path.isfile(outcomeFileName3_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model3_mecab,
                                     load_Classifier_by_model3_mecab,
                                     outcomeFileName3_mecab)
    del token_mecab
    del commentsDf, extCommentsDf

5a29c445588c132954d1973a


In [23]:
warnings.filterwarnings('ignore')
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')
for idx in reDaumData.index:
    clear_output()
    data = reDaumData.loc[idx]
    newsId = data.name
    print (newsId)
    siteInfo = data.site
    if os.path.isdir(os.path.join(outcomeDaumCommentsPath, newsId)):
        outcomePath = os.path.join(outcomeDaumCommentsPath, newsId)
    else:
        os.mkdir(os.path.join(outcomeDaumCommentsPath, newsId))
        outcomePath = os.path.join(outcomeDaumCommentsPath, newsId)
    commentsFile = daumCommentsPath + newsId + '.csv'
    commentsDf = pd.read_csv(commentsFile, encoding='utf-8', header=0)
    commentsDf.dropna(inplace = True)
    extCommentsDf = commentsDf.loc[:,['category', 'comments', 'date', 'rank', 'site', '공감', '비공감']]
    # twitter
    token_ct = commentsDf.apply(lambda x: bm.TokenizeAndTag(ct, x, stopwords, TaggedDocument), axis = 1)
    outcomeFileName1_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name1_ct))
    if not os.path.isfile(outcomeFileName1_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model1_ct,
                                     load_Classifier_by_model1_ct,
                                     outcomeFileName1_ct)
    outcomeFileName2_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name2_ct))
    if not os.path.isfile(outcomeFileName2_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model2_ct,
                                     load_Classifier_by_model2_ct,
                                     outcomeFileName2_ct)
    outcomeFileName3_ct = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name3_ct))
    if not os.path.isfile(outcomeFileName3_ct):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_ct, 
                                     model3_ct,
                                     load_Classifier_by_model3_ct,
                                     outcomeFileName3_ct)
    del token_ct
    # mecab
    token_mecab = commentsDf.apply(lambda x: bm.TokenizeAndTag(mecab, x, stopwords, TaggedDocument), axis = 1)
    outcomeFileName1_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name1_mecab))
    if not os.path.isfile(outcomeFileName1_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model1_mecab,
                                     load_Classifier_by_model1_mecab,
                                     outcomeFileName1_mecab)
    outcomeFileName2_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name2_mecab))
    if not os.path.isfile(outcomeFileName2_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model2_mecab,
                                     load_Classifier_by_model2_mecab,
                                     outcomeFileName2_mecab)
    outcomeFileName3_mecab = os.path.join(outcomePath, 'outcome_comments_sentiment_{}_{}'.format(siteInfo, name3_mecab))
    if not os.path.isfile(outcomeFileName3_mecab):
        PipeLineForSentimentUsingDoc2Vec(extCommentsDf, token_mecab, 
                                     model3_mecab,
                                     load_Classifier_by_model3_mecab,
                                     outcomeFileName3_mecab)
    del token_mecab
    del commentsDf, extCommentsDf
    

5a2a61bf588c13481c229d1e
